In [45]:
import os
import glob
import polars as pl
import pandas as pd
import plotly.express as px

from utils import hermes_download_client
from utils import ambient_parameter_conversion as apc
from utils import calibration_processing as cp

DATA_DIRECTORY = os.environ.get("DATA_DIRECTORY")
PICARRO_DATA_DIRECTORY = os.environ.get("PICARRO_DATA_DIRECTORY")

sensor_id = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20]

# customize pipeline
download_files = False
merge_picarro_files = False

# Download to local db

In [46]:
# download from hermes database
# Use Download/download_from_hermes notebook
if download_files:
    
    while(True):
        try:
            component = hermes_download_client.Extract()
            result = component.execute()
            break
        except Exception as e:
            print(e)

# Process Picarro Data

In [47]:
if merge_picarro_files:
    filenames = glob.glob(PICARRO_DATA_DIRECTORY + "/*/*/*.dat")

    # read all *.dat picarro measurement files and add to single db
    df_list = []
    for filename in filenames:
        df_list.append(pd.read_csv(filename,sep='\s+'))

    df_p_files = pd.concat(df_list, ignore_index=True)
    df_p_files["datetime"] = pd.to_datetime((df_p_files['DATE'] + ' ' + df_p_files['TIME']))
    df_p_files.sort_values(by='datetime', inplace = True)

    df_p_files.to_parquet(path = os.path.join(DATA_DIRECTORY, "input", "picarro.parquet"))

    #Calibration

    # TODO: Add ability for multiple calibration dates
    # before 23.10
    # picarro_slope = 1.0061589132696314
    # picarro_intercept = 0.14607153970888476

    # after 23.10
    #picarro_slope = 1.0063874771746113
    #picarro_intercept = 0.06621464961165202
    
     
    #after 18.12
    picarro_slope = 1.0060713120261249
    picarro_intercept = 0.08088569875155827
    
    #1m averaged corrected Picarro dataset
    df_p_1m = pl.scan_parquet(os.path.join(DATA_DIRECTORY,"input", "picarro.parquet")) \
        .with_columns(pl.col("datetime").dt.cast_time_unit("us").dt.replace_time_zone("UTC").alias("creation_timestamp")) \
        .sort("creation_timestamp") \
        .with_columns((pl.col("CO2_dry") * picarro_slope + picarro_intercept).alias("CO2_corr")) \
        .group_by_dynamic("creation_timestamp", every='1m') \
        .agg(pl.all().exclude("creation_timestamp").mean()).collect() \
        .select(["creation_timestamp", "CO2_corr", "h2o_reported"]) \
        .with_columns([pl.lit(picarro_slope).alias("slope"),
            pl.lit(picarro_intercept).alias("intercept"),
            pl.lit("Picarro").alias("sys_name_short"),
            pl.lit(0.0).alias("diff")]
            ) 

    df_p_1m.write_parquet(os.path.join(DATA_DIRECTORY,"processed", "1m_cal_corr_picarro.parquet"))
    
    
    #1h averaged corrected Picarro dataset
    df_p_10m = df_p_1m.sort("creation_timestamp") \
        .group_by_dynamic("creation_timestamp", every='10m') \
        .agg(pl.all().exclude("creation_timestamp").mean())
        
    df_p_10m.write_parquet(os.path.join(DATA_DIRECTORY,"processed", "10m_cal_corr_picarro.parquet"))
    
    #1h averaged corrected Picarro dataset
    df_p_1h = df_p_1m.sort("creation_timestamp") \
        .group_by_dynamic("creation_timestamp", every='1h') \
        .agg(pl.all().exclude("creation_timestamp").mean())
        
    df_p_1h.write_parquet(os.path.join(DATA_DIRECTORY,"processed", "1h_cal_corr_picarro.parquet"))

# Import data files

In [48]:
# load calibration bottle concentrations (preprocessed)
df_gas = pl.read_csv(os.path.join(DATA_DIRECTORY,"input", "averaged_gases.csv"))
# load local db: acropolis.parquet
df_raw = pl.scan_parquet(os.path.join(DATA_DIRECTORY, "download", "acropolis.parquet")) \
    .filter(pl.col("system_name") != "test-sensor") \
    .with_columns(pl.col("system_name").str.extract(r'(\d+)',1).str.to_integer().alias("system_id"))
    
df_p_1m = pl.read_parquet(os.path.join(DATA_DIRECTORY,"processed", "1m_cal_corr_picarro.parquet"))  

# Perform Dry-Wet Conversion

### Measurement Data

In [49]:
# aggregate data to 1m
df_1_m = df_raw.sort("creation_timestamp") \
    .select(pl.all().exclude('^wxt532_.*$', '^cal_.*$', '^enclosure_.*$', '^raspi_.*$', '^ups_.*$')) \
    .filter(pl.col('gmp343_filtered') > 0) \
    .filter(pl.col('gmp343_temperature') > 0) \
    .filter(pl.col('sht45_humidity') > 0) \
    .filter(pl.col('bme280_pressure') > 0) \
    .group_by_dynamic("creation_timestamp", every='1m', by= "system_id")  \
    .agg(pl.all().exclude(["creation_timestamp","system_id"]).mean()) \
    
# perform dry conversion for measurement data                
df_1_m = df_1_m.with_columns(pl.struct(['gmp343_temperature','sht45_humidity']) \
    .map_elements(lambda x: apc.rh_to_ah(x['sht45_humidity'],apc.absolute_temperature(x['gmp343_temperature'])))
    .alias("h2o_ah")) \
    .with_columns(pl.struct(['gmp343_temperature','sht45_humidity','bme280_pressure'])
    .map_elements(lambda x: (apc.rh_to_molar_mixing(x['sht45_humidity'],apc.absolute_temperature(x['gmp343_temperature']),x['bme280_pressure']*100))*100) \
    .alias("h2o_v%")) \
    .with_columns(pl.struct(['gmp343_filtered','gmp343_temperature','sht45_humidity','bme280_pressure']) \
    .map_elements(lambda x: apc.calculate_co2dry(x['gmp343_filtered'],x['gmp343_temperature'],x['sht45_humidity'],x['bme280_pressure']*100))
    .alias("gmp343_dry")).collect()

In [50]:
df_1_m.tail(3).select("creation_timestamp","system_id","gmp343_filtered", "h2o_ah", "h2o_v%" ,"gmp343_dry")

creation_timestamp,system_id,gmp343_filtered,h2o_ah,h2o_v%,gmp343_dry
"datetime[μs, UTC]",i64,f64,f64,f64,f64
2024-07-15 15:12:00 UTC,17,422.183333,15.688842,2.396899,432.551148
2024-07-15 15:13:00 UTC,17,420.983333,15.729823,2.410124,431.380131
2024-07-15 15:14:00 UTC,17,420.35,15.768284,2.406386,430.714655


In [51]:
#df_1_m.write_parquet(os.path.join(DATA_DIRECTORY, "processed", "1m_acropolis_dry.parquet"))

### Calibration Data

In [52]:
df_dry_calibration = df_raw.filter(pl.col("cal_gmp343_filtered") > 0) \
    .filter(pl.col("cal_gmp343_temperature") > 0) \
    .filter(pl.col("cal_bme280_pressure") > 0) \
    .with_columns(pl.col("cal_sht45_humidity").fill_null(0.0)) \
    .with_columns(pl.struct(['cal_gmp343_temperature','cal_sht45_humidity','cal_bme280_pressure'])
    .map_elements(lambda x: apc.rh_to_molar_mixing(x['cal_sht45_humidity'],apc.absolute_temperature(x['cal_gmp343_temperature']),x['cal_bme280_pressure']*100)) \
    .alias("cal_h2o_v%")) \
    .with_columns(pl.struct(['cal_gmp343_filtered','cal_gmp343_temperature','cal_sht45_humidity','cal_bme280_pressure']) \
    .map_elements(lambda x: apc.calculate_co2dry(x['cal_gmp343_filtered'],x['cal_gmp343_temperature'],x['cal_sht45_humidity'],x['cal_bme280_pressure']*100))
    .alias("cal_gmp343_dry")) \
    .select("creation_timestamp","system_id", '^cal_.*$') \
    .filter((pl.col("cal_bottle_id") > 0) & (pl.col("cal_bottle_id") <= df_gas["cal_bottle_id"].max())) \
    .collect()

In [53]:
df_dry_calibration.filter(pl.col("system_id")==1).tail(3)

creation_timestamp,system_id,cal_bottle_id,cal_gmp343_raw,cal_gmp343_compensated,cal_gmp343_filtered,cal_gmp343_temperature,cal_bme280_temperature,cal_bme280_humidity,cal_bme280_pressure,cal_sht45_temperature,cal_sht45_humidity,cal_h2o_v%,cal_gmp343_dry
"datetime[μs, UTC]",i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
2024-07-15 03:31:38.260 UTC,1,59.0,381.0,442.6,445.7,36.9,39.09,0.0,947.6,39.11,0.49,0.000323,445.844049
2024-07-15 03:31:48.270 UTC,1,59.0,382.1,444.0,444.0,36.9,39.13,0.0,946.79,39.16,0.44,0.00029,444.128963
2024-07-15 03:31:58.290 UTC,1,59.0,382.0,444.0,446.2,36.9,39.17,0.0,946.32,39.21,0.48,0.000317,446.341458


# Calculate Calibration Correction

In [81]:
df_slope_intercept = df_dry_calibration.join(df_gas.cast({"cal_bottle_id": pl.Float64}), on = ["cal_bottle_id"], how= "left") \
    .with_columns((pl.col("creation_timestamp").dt.date()).alias("date")) \
    .sort("date") \
    .group_by([pl.col("date"), pl.col("system_id"), pl.col("cal_bottle_id")]) \
    .agg([
        pl.col("cal_gmp343_dry"),
        pl.col("cal_bottle_CO2").last(),
        pl.col("creation_timestamp").last(),
        ]) \
    .with_columns([pl.col("cal_gmp343_dry").map_elements(lambda x: cp.process_bottle(x))]) \
    .filter(pl.col("cal_gmp343_dry") > 0) \
    .sort(pl.col("cal_gmp343_dry")) \
    .group_by(["date", "system_id"]) \
    .agg([
        pl.col("cal_gmp343_dry"),
        pl.col("cal_bottle_CO2"),
        pl.col("creation_timestamp").last()
        ]) \
    .filter(pl.col("cal_gmp343_dry").list.len() == 2) \
    .with_columns(pl.struct(['cal_gmp343_dry','cal_bottle_CO2']) \
    .map_elements(lambda x: cp.two_point_calibration(x['cal_gmp343_dry'],x['cal_bottle_CO2'])) \
    .alias('slope, intercept')) \
    .with_columns([(pl.col("slope, intercept").list.first()).alias("slope"),
                   (pl.col("slope, intercept").list.last()).alias("intercept")
                   ]) \
    .select("creation_timestamp", "date", "system_id", "slope", "intercept") \
    .filter(pl.col("slope") > 0)

In [82]:
df_slope_intercept.head(3)

creation_timestamp,date,system_id,slope,intercept
"datetime[μs, UTC]",date,i64,f64,f64
2023-08-02 09:32:07.320 UTC,2023-08-02,6,1.03987,15.979988
2023-08-10 17:21:28.810 UTC,2023-08-10,6,1.034451,8.612635
2023-07-12 13:20:06.680 UTC,2023-07-12,6,1.033199,8.377939


In [84]:
# safe results to parquet
df_slope_intercept.write_parquet(os.path.join(DATA_DIRECTORY, "processed", "slope_intercept_acropolis.parquet"))

In [83]:
fig = px.scatter(df_slope_intercept.filter((pl.col("slope") > 0.7) & (pl.col("slope") < 1.1)), x="creation_timestamp", y = "slope", color = "system_id")
fig.show()
fig = px.histogram(df_slope_intercept.filter((pl.col("slope") > 0.7) & (pl.col("slope") < 1.1)), x="slope", color = "system_id")
fig.show()
fig = px.histogram(df_slope_intercept.filter((pl.col("intercept") < 100) & (pl.col("intercept") > -100)), x="intercept", color = "system_id")
fig.show()

# Perform Calibration Correction

## 1m aggregated data

In [58]:
df_1_m = df_1_m.with_columns(pl.struct(["system_id"]) \
    .map_elements(lambda x: f"acropolis-{x['system_id']}") \
    .alias("sys_name_short")) \
    .drop("system_name")

In [59]:
df_wind = df_raw.select(pl.col("^(wxt532_.*|creation_timestamp|system_id)$")).filter(pl.col('wxt532_direction_avg') > 0).collect()

In [60]:
df_systems = []

for id in sensor_id:
    df_system_calibration = df_slope_intercept.filter(pl.col("system_id") == id) \
        .sort("creation_timestamp") \
        .drop("system_id", "system_name", "date")
    
    df_system = df_1_m.filter(pl.col("system_id") == id) \
        .sort("creation_timestamp") \
        .join_asof(df_system_calibration, on="creation_timestamp", strategy="nearest", tolerance="10m") \
        .join_asof(df_wind.filter(pl.col("system_id") == id).sort("creation_timestamp"), on="creation_timestamp", strategy="nearest", tolerance="10m") \
        .join(df_p_1m.select("creation_timestamp", "CO2_corr"), on = ["creation_timestamp"], how= "left") \
        .with_columns([
            pl.col("slope").interpolate(),
            pl.col("intercept").interpolate()
            ]) \
        .with_columns([
            pl.col("slope").forward_fill(),
            pl.col("intercept").forward_fill()
            ]) \
        .with_columns(((pl.col("gmp343_dry")) * pl.col("slope") + pl.col("intercept")).alias("gmp343_corrected")) \
        .with_columns((pl.col("CO2_corr") - pl.col("gmp343_corrected")).alias("diff")) \
        .with_columns((pl.col("creation_timestamp").dt.date()).alias("date"))
    
    df_systems.append(df_system)
        

df_1_m = pl.concat(df_systems, how="diagonal")

In [61]:
df_1_m.write_parquet(os.path.join(DATA_DIRECTORY, "processed", "1m_cal_corr_acropolis.parquet"))

## 10m aggregated data

In [62]:
df_10_m = df_1_m.sort("creation_timestamp") \
        .group_by_dynamic("creation_timestamp", every='10m', by=["system_id", "sys_name_short"]) \
        .agg(pl.all().exclude(["creation_timestamp","sys_name_short"]).mean())
        
df_10_m.write_parquet(os.path.join(DATA_DIRECTORY, "processed", "10m_cal_corr_acropolis.parquet"))

## 1h aggregated data

In [63]:
# TODO: implement despiking algorithm

In [64]:
df_1_h = df_1_m.sort("creation_timestamp") \
        .group_by_dynamic("creation_timestamp", every='1h', by=["system_id", "sys_name_short"]) \
        .agg(pl.all().exclude(["creation_timestamp","sys_name_short"]).mean())
        
df_1_h.write_parquet(os.path.join(DATA_DIRECTORY, "processed", "1h_cal_corr_acropolis.parquet"))

# Plot data

In [65]:
df_plot_1 = df_p_1m[["creation_timestamp", "sys_name_short", "CO2_corr"]].rename({"CO2_corr": "CO2"})

In [66]:
df_plot_2 = df_1_m[["creation_timestamp", "sys_name_short", "gmp343_corrected"]].rename({"gmp343_corrected": "CO2"})

In [67]:
df_plot = pl.concat([df_plot_1,df_plot_2])

In [68]:
# Quality Check

from datetime import datetime, timezone

start = datetime(2024, 3, 16, 10, 0, 0).replace(tzinfo=timezone.utc)
end = datetime(2024, 3, 16, 22, 0, 0).replace(tzinfo=timezone.utc)

df_temp = df_plot.filter(pl.col("creation_timestamp").is_between(start, end)).filter(pl.col("sys_name_short").is_in(["acropolis-7", "Picarro"]))

fig = px.scatter(df_temp, x="creation_timestamp", y = "CO2", color = "sys_name_short")
fig.show()